In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
%pylab inline

import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import pandasql as ps



import statsmodels.formula.api as smf

Populating the interactive namespace from numpy and matplotlib


#link to tableau visualizations as of 12/20/19

# https://public.tableau.com/profile/angelyn.arthur#!/vizhome/home_credit_default_risk_project/CountbyGender?publish=yes

In [2]:
application_test = pd.read_csv('application_test.csv')

In [5]:
application_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
application_train = pd.read_csv('application_train.csv')

In [7]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [22]:
missing_data(application_train).head(40)

,Total,Percent
COMMONAREA_MEDI,214865,69.872297
COMMONAREA_AVG,214865,69.872297
COMMONAREA_MODE,214865,69.872297
NONLIVINGAPARTMENTS_MODE,213514,69.432963
NONLIVINGAPARTMENTS_MEDI,213514,69.432963
NONLIVINGAPARTMENTS_AVG,213514,69.432963
FONDKAPREMONT_MODE,210295,68.386172
LIVINGAPARTMENTS_MEDI,210199,68.354953
LIVINGAPARTMENTS_MODE,210199,68.354953
LIVINGAPARTMENTS_AVG,210199,68.354953


In [11]:
pd.set_option('display.max_columns', None)

In [4]:
list(application_train.columns)

['SK_ID_CURR',
 'TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_A

In [34]:


test_query = "SELECT FLAG_DOCUMENT_2, FLAG_DOCUMENT_3 FROM application_train"
q = ps.sqldf(test_query,locals())
q.head(20)

,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3
0,0,1
1,0,1
2,0,0
3,0,1
4,0,0
5,0,1
6,0,0
7,0,1
8,0,1
9,0,0


In [8]:
contract_query = "SELECT NAME_CONTRACT_TYPE FROM application_train"
q1 = ps.sqldf(contract_query,locals())
q1.head(20)

,NAME_CONTRACT_TYPE
0,Cash loans
1,Cash loans
2,Revolving loans
3,Cash loans
4,Cash loans
5,Cash loans
6,Cash loans
7,Cash loans
8,Cash loans
9,Revolving loans


In [11]:
# Number of unique classes in each object column
application_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
WALLSMATERIAL_MODE             7
EMERGENCYSTATE_MODE            2
dtype: int64

In [10]:
income_type_query = "SELECT distinct NAME_INCOME_TYPE FROM application_train"
q2 = ps.sqldf(income_type_query,locals())
q2.head(20)

,NAME_INCOME_TYPE
0,Working
1,State servant
2,Commercial associate
3,Pensioner
4,Unemployed
5,Student
6,Businessman
7,Maternity leave


In [15]:
application_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [28]:
#new dataset with only variables want to use (will amend, have amended so far)

application_train_new = application_train[['SK_ID_CURR','TARGET','NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR',
 'FLAG_OWN_REALTY','CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION','OCCUPATION_TYPE','CNT_FAM_MEMBERS']]
application_train_new.head() 

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,Laborers,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,Core staff,2.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,Laborers,1.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,Laborers,2.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,Core staff,1.0


In [31]:
#TO EXPORT TO CSV

application_train_new.to_csv('application_train_new_df.csv')

In [20]:
# 2/3 of all values are null in own_car_age, dropped column. 
q3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 1 columns):
OWN_CAR_AGE    104582 non-null float64
dtypes: float64(1)
memory usage: 2.3 MB


In [21]:
q3.isnull().sum().sum()

202929

In [23]:
#TABLE WITH ALL VARIABLES IN QUERY, ONLY AGE CONVERTED TO YRS FOR NOW, 

all_variables_query = "SELECT SK_ID_CURR, TARGET, NAME_CONTRACT_TYPE, CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY, CNT_CHILDREN, AMT_INCOME_TOTAL, AMT_CREDIT, AMT_ANNUITY, AMT_GOODS_PRICE, NAME_TYPE_SUITE, NAME_INCOME_TYPE, NAME_EDUCATION_TYPE, NAME_FAMILY_STATUS, NAME_HOUSING_TYPE, REGION_POPULATION_RELATIVE, DAYS_BIRTH,(days_birth/-365) as age_in_years, DAYS_EMPLOYED, DAYS_REGISTRATION, OCCUPATION_TYPE, CNT_FAM_MEMBERS from application_train_new"
all_var_df = ps.sqldf(all_variables_query,locals())
all_var_df.head(20)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,age_in_years,DAYS_EMPLOYED,DAYS_REGISTRATION,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,100002,1,Cash loans,M,N,Y,0,202500.000,406597.5,24700.5,...,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,25,-637,-3648.0,Laborers,1.0
1,100003,0,Cash loans,F,N,N,0,270000.000,1293502.5,35698.5,...,Higher education,Married,House / apartment,0.003541,-16765,45,-1188,-1186.0,Core staff,2.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.000,135000.0,6750.0,...,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,52,-225,-4260.0,Laborers,1.0
3,100006,0,Cash loans,F,N,Y,0,135000.000,312682.5,29686.5,...,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,52,-3039,-9833.0,Laborers,2.0
4,100007,0,Cash loans,M,N,Y,0,121500.000,513000.0,21865.5,...,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,54,-3038,-4311.0,Core staff,1.0
5,100008,0,Cash loans,M,N,Y,0,99000.000,490495.5,27517.5,...,Secondary / secondary special,Married,House / apartment,0.035792,-16941,46,-1588,-4970.0,Laborers,2.0
6,100009,0,Cash loans,F,Y,Y,1,171000.000,1560726.0,41301.0,...,Higher education,Married,House / apartment,0.035792,-13778,37,-3130,-1213.0,Accountants,3.0
7,100010,0,Cash loans,M,Y,Y,0,360000.000,1530000.0,42075.0,...,Higher education,Married,House / apartment,0.003122,-18850,51,-449,-4597.0,Managers,2.0
8,100011,0,Cash loans,F,N,Y,0,112500.000,1019610.0,33826.5,...,Secondary / secondary special,Married,House / apartment,0.018634,-20099,55,365243,-7427.0,None,2.0
9,100012,0,Revolving loans,M,N,Y,0,135000.000,405000.0,20250.0,...,Secondary / secondary special,Single / not married,House / apartment,0.019689,-14469,39,-2019,-14437.0,Laborers,1.0


In [33]:
all_var_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 23 columns):
SK_ID_CURR                    307511 non-null int64
TARGET                        307511 non-null int64
NAME_CONTRACT_TYPE            307511 non-null object
CODE_GENDER                   307511 non-null object
FLAG_OWN_CAR                  307511 non-null object
FLAG_OWN_REALTY               307511 non-null object
CNT_CHILDREN                  307511 non-null int64
AMT_INCOME_TOTAL              307511 non-null float64
AMT_CREDIT                    307511 non-null float64
AMT_ANNUITY                   307499 non-null float64
AMT_GOODS_PRICE               307233 non-null float64
NAME_TYPE_SUITE               306219 non-null object
NAME_INCOME_TYPE              307511 non-null object
NAME_EDUCATION_TYPE           307511 non-null object
NAME_FAMILY_STATUS            307511 non-null object
NAME_HOUSING_TYPE             307511 non-null object
REGION_POPULATION_RELATIVE    307511 non

In [32]:
#CAN EXPORT TO CSV WITH QUERIES, ONLY WITH AGE SO FAR

all_var_df.to_csv('all_var_df_age_calc.csv')

In [21]:
#note** will have to do /-365 in application_train_new df directly to use it for visulizations
#in tableau when exporting as csv

days_birth_query = "SELECT target, days_birth, (days_birth/-365) as age_in_years from application_train_new"
q4 = ps.sqldf(days_birth_query,locals())
q4.head(20)

,TARGET,DAYS_BIRTH,age_in_years
0,1,-9461,25
1,0,-16765,45
2,0,-19046,52
3,0,-19005,52
4,0,-19932,54
5,0,-16941,46
6,0,-13778,37
7,0,-18850,51
8,0,-20099,55
9,0,-14469,39


In [16]:
q4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 3 columns):
TARGET        307511 non-null int64
DAYS_BIRTH    307511 non-null int64
age           307511 non-null int64
dtypes: int64(3)
memory usage: 7.0 MB


In [12]:
#query to see the difference in number of entries when 365243 is taken out
days_employed_queryt = "SELECT days_employed, (days_employed/-365) from application_train_new"
q5t = ps.sqldf(days_employed_queryt,locals())
q5t.head(20)

,DAYS_EMPLOYED,(days_employed/-365)
0,-637,1
1,-1188,3
2,-225,0
3,-3039,8
4,-3038,8
5,-1588,4
6,-3130,8
7,-449,1
8,365243,-1000
9,-2019,5


In [13]:
q5t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 2 columns):
DAYS_EMPLOYED           307511 non-null int64
(days_employed/-365)    307511 non-null int64
dtypes: int64(2)
memory usage: 4.7 MB


In [9]:
#how to handle days employed column? (column is correct, in yrs, issue with non negative value)
#value of 365243 taken out since outlier(?). number of entries now  252136 instead of 307510, around 55,000 difference.

days_employed_query = "SELECT days_employed, (days_employed/-365) from application_train_new where days_employed NOT IN (365243)"
q5 = ps.sqldf(days_employed_query,locals())
q5.head(20)

,DAYS_EMPLOYED,(days_employed/-365)
0,-637,1
1,-1188,3
2,-225,0
3,-3039,8
4,-3038,8
5,-1588,4
6,-3130,8
7,-449,1
8,-2019,5
9,-679,1


In [10]:
q5.describe()

,DAYS_EMPLOYED,(days_employed/-365)
count,252137.000000,252137.000000
mean,-2384.169325,6.037008
std,2338.360162,6.416079
min,-17912.000000,0.000000
25%,-3175.000000,2.000000
50%,-1648.000000,4.000000
75%,-767.000000,8.000000
max,0.000000,49.000000


In [11]:
q5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252137 entries, 0 to 252136
Data columns (total 2 columns):
DAYS_EMPLOYED           252137 non-null int64
(days_employed/-365)    252137 non-null int64
dtypes: int64(2)
memory usage: 3.8 MB


In [27]:
#registration in years, do more research if loans can be taken out for 20+ yrs etc for example

days_registration_query = "SELECT days_registration, (days_registration/-365) as in_years from new_dataset"
q6 = ps.sqldf(days_registration_query,locals())
q6.head(20)

,DAYS_REGISTRATION,in_years
0,-3648.0,9.994521
1,-1186.0,3.249315
2,-4260.0,11.671233
3,-9833.0,26.939726
4,-4311.0,11.810959
5,-4970.0,13.616438
6,-1213.0,3.323288
7,-4597.0,12.594521
8,-7427.0,20.347945
9,-14437.0,39.553425


In [33]:
q6.describe()

,DAYS_REGISTRATION,in_years
count,307511.000000,307511.000000
mean,-4986.120328,13.660604
std,3522.886321,9.651743
min,-24672.000000,-0.000000
25%,-7479.500000,5.506849
50%,-4504.000000,12.339726
75%,-2010.000000,20.491781
max,0.000000,67.594521


In [5]:
application_train.isnull().sum().sum()

9152465

In [8]:
application_train.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


In [2]:
bureau = pd.read_csv('bureau.csv')

In [4]:
#add columns from bureau**, or not sincce this is from previous data of clients in other banks

bureau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
SK_ID_CURR                int64
SK_ID_BUREAU              int64
CREDIT_ACTIVE             object
CREDIT_CURRENCY           object
DAYS_CREDIT               int64
CREDIT_DAY_OVERDUE        int64
DAYS_CREDIT_ENDDATE       float64
DAYS_ENDDATE_FACT         float64
AMT_CREDIT_MAX_OVERDUE    float64
CNT_CREDIT_PROLONG        int64
AMT_CREDIT_SUM            float64
AMT_CREDIT_SUM_DEBT       float64
AMT_CREDIT_SUM_LIMIT      float64
AMT_CREDIT_SUM_OVERDUE    float64
CREDIT_TYPE               object
DAYS_CREDIT_UPDATE        int64
AMT_ANNUITY               float64
dtypes: float64(8), int64(6), object(3)
memory usage: 222.6+ MB
